In [1]:
from nltk.translate import bleu
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu, SmoothingFunction
from modelT5 import T5Model
import os
import argparse
from transformers import T5Tokenizer, T5ForConditionalGeneration
import pandas as pd
import numpy as np
from datasets import Dataset
import tensorflow as tf
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorForSeq2Seq
from transformers import (
    # AdamW, 
    T5Model, 
    T5ForConditionalGeneration, 
    T5AdapterModel, 
    T5Tokenizer, 
    get_linear_schedule_with_warmup,
    TrainingArguments, 
    AdapterTrainer,
    Trainer
)
import torch

[nltk_data] Downloading package punkt to /Users/tyler/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/Users/tyler/opt/anaconda3/envs/cs-7643/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df_test = pd.read_csv('data/COMBINED/test.tsv', sep = '\t', names=['in', 'expected'])
df_test = df_test.sample(n=1500, replace=False, random_state=42, ignore_index=True)
model = T5AdapterModel.from_pretrained("t5-small")
adapter_name = model.load_adapter("custom_adapters/houlsby-inv-sm")
model.set_active_adapters(adapter_name)
tokenizer = T5Tokenizer.from_pretrained('t5-small')
print(model.active_adapters, adapter_name)

Some weights of the model checkpoint at t5-small were not used when initializing T5AdapterModel: ['decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight']
- This IS expected if you are initializing T5AdapterModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5AdapterModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of T5AdapterModel were not initialized from the model checkpoint at t5-small and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Stack[paraphrase] paraphrase


In [3]:
df_test['sentence_inputs'] = df_test.apply(lambda x: x['in'].split(), axis=1)
df_test['sentence_length'] = df_test.apply(lambda x: len(x['sentence_inputs']), axis=1)
df_test['to_model'] = df_test.apply(lambda x: 'paraphrase: ' + x['in'], axis=1)
df_test['encoding'] = df_test.apply(lambda x: tokenizer(x['in'], return_tensors="tf", truncation=True), axis=1)
df_test['input_ids'] = df_test.apply(lambda x: x['encoding']['input_ids'], axis=1)
df_test['attention_mask'] = df_test.apply(lambda x: x['encoding']['attention_mask'], axis=1)
# def model_func(x):
#     return model.generate(
#         input_ids=x['encoding']['input_ids'],
#         do_sample=True,
#         attention_mask=x['encoding']['attention_mask'],
#         max_length=512, top_k=250, top_p=0.99, early_stopping=True, num_return_sequences=2)
# df_test['out'] = df_test.apply(lambda x: model_func(x), axis=1)
df_test = df_test[df_test['sentence_length'] < 30]
df_test.head()

2022-04-29 16:19:52.869572: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


,in,expected,sentence_inputs,sentence_length,to_model,encoding,input_ids,attention_mask
0,Lillian `` Lilly '' Truscott ( Oliver Oken ) i...,Lillian `` Lilly '' Truscott ( Oliver Oken ) i...,"[Lillian, ``, Lilly, '', Truscott, (, Oliver, ...",27,paraphrase: Lillian `` Lilly '' Truscott ( Oli...,"[input_ids, attention_mask]","((tf.Tensor(301, shape=(), dtype=int32), tf.Te...","((tf.Tensor(1, shape=(), dtype=int32), tf.Tens..."
1,Alfred Matthew Hubbard introduced Aldous Huxle...,"In 1955 , Alfred Matthew Hubbard introduced Al...","[Alfred, Matthew, Hubbard, introduced, Aldous,...",20,paraphrase: Alfred Matthew Hubbard introduced ...,"[input_ids, attention_mask]","((tf.Tensor(19850, shape=(), dtype=int32), tf....","((tf.Tensor(1, shape=(), dtype=int32), tf.Tens..."
2,There were several competing organisations on ...,"During this period , there have been several c...","[There, were, several, competing, organisation...",17,paraphrase: There were several competing organ...,"[input_ids, attention_mask]","((tf.Tensor(290, shape=(), dtype=int32), tf.Te...","((tf.Tensor(1, shape=(), dtype=int32), tf.Tens..."
3,In the belief that she is Easwari Rao ( Kasthu...,Thinking that she is Kasthuri ( Easwari Rao ) ...,"[In, the, belief, that, she, is, Easwari, Rao,...",23,paraphrase: In the belief that she is Easwari ...,"[input_ids, attention_mask]","((tf.Tensor(86, shape=(), dtype=int32), tf.Ten...","((tf.Tensor(1, shape=(), dtype=int32), tf.Tens..."
4,"Tomasz Kafarski ( born January 13 , 1975 in Ko...","Tomasz Kafarski ( born January 13 , 1975 in Ko...","[Tomasz, Kafarski, (, born, January, 13, ,, 19...",21,paraphrase: Tomasz Kafarski ( born January 13 ...,"[input_ids, attention_mask]","((tf.Tensor(3059, shape=(), dtype=int32), tf.T...","((tf.Tensor(1, shape=(), dtype=int32), tf.Tens..."


In [4]:
full_outs = []
inps = tokenizer(df_test['to_model'].values.tolist(),
    padding=True, truncation=True, return_tensors="pt")
# new_inps = model.prepare_inputs_for_generation(input_ids=inps['input_ids'],
#     do_sample=True, attention_mask=inps['attention_mask'], max_length=512,
#     top_k=250, top_p=0.99, early_stopping=True, num_return_sequences=2)
# new_inps.keys()
inps['input_ids'].shape

torch.Size([1436, 63])

In [5]:
full_outs = []

In [6]:
out = model.generate(
    input_ids=inps['input_ids'],
    attention_mask=inps['attention_mask'],
    top_k=250, top_p=0.99, early_stopping=True, num_return_sequences=3, do_sample=True)
full_outs.append(out)

In [7]:
sentence_outputs = np.reshape(
    [x.split() for x in tokenizer.batch_decode(out, skip_special_tokens=True)],
    (df_test.shape[0], -1))
to_df = [list(x) for x in sentence_outputs] # allows us to put all generated sentences in 1 column in list form
df_test['out'] = to_df
df_test.head()

/Users/tyler/opt/anaconda3/envs/cs-7643/lib/python3.7/site-packages/numpy/core/fromnumeric.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = getattr(asarray(obj), method)(*args, **kwds)


,in,expected,sentence_inputs,sentence_length,to_model,encoding,input_ids,attention_mask,out
0,Lillian `` Lilly '' Truscott ( Oliver Oken ) i...,Lillian `` Lilly '' Truscott ( Oliver Oken ) i...,"[Lillian, ``, Lilly, '', Truscott, (, Oliver, ...",27,paraphrase: Lillian `` Lilly '' Truscott ( Oli...,"[input_ids, attention_mask]","((tf.Tensor(301, shape=(), dtype=int32), tf.Te...","((tf.Tensor(1, shape=(), dtype=int32), tf.Tens...","[[Lillian, Lilly, '], [], [Lillian, Lilly, '',..."
1,Alfred Matthew Hubbard introduced Aldous Huxle...,"In 1955 , Alfred Matthew Hubbard introduced Al...","[Alfred, Matthew, Hubbard, introduced, Aldous,...",20,paraphrase: Alfred Matthew Hubbard introduced ...,"[input_ids, attention_mask]","((tf.Tensor(19850, shape=(), dtype=int32), tf....","((tf.Tensor(1, shape=(), dtype=int32), tf.Tens...","[[Alfred, Matthew, Hubbard, introduced, Ald], ..."
2,There were several competing organisations on ...,"During this period , there have been several c...","[There, were, several, competing, organisation...",17,paraphrase: There were several competing organ...,"[input_ids, attention_mask]","((tf.Tensor(290, shape=(), dtype=int32), tf.Te...","((tf.Tensor(1, shape=(), dtype=int32), tf.Tens...","[[In, spit’, on, Irish, side, during, the, per..."
3,In the belief that she is Easwari Rao ( Kasthu...,Thinking that she is Kasthuri ( Easwari Rao ) ...,"[In, the, belief, that, she, is, Easwari, Rao,...",23,paraphrase: In the belief that she is Easwari ...,"[input_ids, attention_mask]","((tf.Tensor(86, shape=(), dtype=int32), tf.Ten...","((tf.Tensor(1, shape=(), dtype=int32), tf.Tens...","[[], [In, the, belief, she, is, Easwari, Rao, ..."
4,"Tomasz Kafarski ( born January 13 , 1975 in Ko...","Tomasz Kafarski ( born January 13 , 1975 in Ko...","[Tomasz, Kafarski, (, born, January, 13, ,, 19...",21,paraphrase: Tomasz Kafarski ( born January 13 ...,"[input_ids, attention_mask]","((tf.Tensor(3059, shape=(), dtype=int32), tf.T...","((tf.Tensor(1, shape=(), dtype=int32), tf.Tens...","[[Tomasz, Kafarski, (, born, January, 13,, 197..."


In [8]:
sentence_inputs = [x.split() for x in df_test['in'].values.tolist()]
sentence_expected = [x.split() for x in df_test['expected'].values.tolist()]
sentence_outputs = df_test['out'].values.tolist()
input_bleu = corpus_bleu(sentence_outputs, sentence_inputs)
exp_bleu = corpus_bleu(sentence_outputs, sentence_expected)
print(input_bleu, exp_bleu)

0.3092555234419589 0.18873935413979945


In [9]:
df_test.to_csv('./results/houlsby-inv_1500_3.tsv', sep='\t')